# Wheel Loader Service Request
This is a KWH HMI which allowes to request the wheel loader's *currentJobsDescription* service. Everyone can log in with their access data and authorize this HMI to call up the wheel loader's *currentJobsDescription* service. If you do not have an account yet, please contact the S³I team (s3i@kwh40.de). Of course you can also log in as one of the provided KWH user - forest expert or forest owner - if you do not want to use your own credentials.

First, all necessary modules are imported into the script, including the S³I library.

In [12]:
import s3i
import json
import uuid
import jwt
import time
import os
import base64
import getpass
import requests

This Notebook acts as a client in terms of the S³I. In order to use the S³I it needs a client id and the respective secret. You can assign this notebook to your personal HMI, to make this notebook your HMI. Therefore enter the id and the secret of your HMI in the following input fields.

In [17]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: Assign a client to this notebook. (The id of your HMI)")
clientId = input('[S3I]: Please enter the client id:')
clientSecret = getpass.getpass('[S3I]: Please enter the secret:')
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: Client id and secret are set")

[S3I][2020-05-23 20:31:25]: Assign a client to this notebook. (The id of your HMI)
[S3I]: Please enter the client id:s3i:a3afa256-b80c-4a6d-bd9c-786887c8ce4a
[S3I]: Please enter the secret:········
[S3I][2020-05-23 20:31:57]: Client id and secret are set


Next, you have to enter your username and password. With your access data a token is requested which authorizes this client (your digital twin) to call up the wheel loader service on your behalf.

In [18]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: DEMO Wheel loader (service call), please log in! (sachverstaendiger)")
username = input('[S3I]: Please enter your username:')
password = getpass.getpass('[S3I]: Please enter the password:')
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                                   ) + "]: Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                         client_id=clientId, client_secret=clientSecret, username=username, password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN, scope="")

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                               time.localtime()) + "]: Token received " + parsed_username + " logged in.")

[S3I][2020-05-23 20:31:59]: DEMO Wheel loader (service call), please log in! (sachverstaendiger)
[S3I]: Please enter your username:gebhard
[S3I]: Please enter the password:········
[S3I][2020-05-23 20:34:12]: Your credentials are sent to S3I IdentityProvider.
[S3I][2020-05-23 20:34:12]: Token received gebhard logged in.


After the token has been received, it can be used to query the endpoint of the wheel loader. To query the wheel loader in the S³I Directory, you must have the rights to view the wheel loader. With the creation of your S³I account you should have got this right. In case of problems please contact the S³I team (s3i@kwh40.de).

Besides the endpoint of the wheel loder, the enpoint of the HMI is also needed to add as "ReplyToEnpoint" field in the request. The endpoint of this HMI is also queried from the directory, as it may has changed. By querying its own endpoint from the directory, the HMI is independent of changes made to its own endpoint. These changes then only need to be listed in the directory.

In [24]:
''' authentication with JWT in S3I Directory 
'''

s3i_directory = s3i.Directory(
    s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)

print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                                   ) + "]: Authentication with Token in S3I Directory successful")
    
''' query the sender's endpoint
'''
sender_endpoint = s3i_directory.queryThingIDBased(clientId+"/attributes/allEndpoints")[0]
    
''' query the wheel loaders's Id and endpoint
'''
wheelloaderID = s3i_directory.queryAttributeBased(
    "name", "wheelloader")[0]["thingId"]

wheelloaderServiceEndpoint = s3i_directory.queryEndpointService(
    wheelloaderID, "currentJobsDescription")

receivers = [wheelloaderID]
receiver_endpoints = wheelloaderServiceEndpoint

print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: Prepare the service request.")

parameters = {
    "service":"No parameters needed"
}
msg_uuid = "s3i:" + str(uuid.uuid4())

'''
create service request
'''
servReq = s3i.ServiceRequest()
servReq.fillServiceRequest(senderUUID=clientId, receiverUUID=receivers, sender_endpoint=sender_endpoint,
                           serviceType="currentJobsDescription", parameters=parameters, msgUUID=msg_uuid)

[S3I][2020-05-23 20:39:28]: Authentication with Token in S3I Directory successful
[S3I][2020-05-23 20:39:28]: Prepare the service request.


After querying the enpoints, the message is prepared. Since it is a service request, a *ServiceRequest* is instantiated and filled in.

In [25]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: Sending the service request to the wheel loader")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
print(receiver_endpoints)
response = requests.post(url="https://broker.s3i.vswf.dev/"+receiver_endpoints[0] ,
                                data=json.dumps(servReq.msg), headers=headers)
print(response.text)

[S3I][2020-05-23 20:39:42]: Sending the service request to the wheel loader
['s3ibs://s3i:7f654f13-11ac-413d-841c-10a8e431fc35']
[{"endpoint": "s3ibs://s3i:7f654f13-11ac-413d-841c-10a8e431fc35", "status": "Sending message succeeded"}]


The HMI uses the token to establish a connection to the broker and sends the service request to the wheel loader's endpoint. It then waits for incoming responses. If a message arrives, the callback function is called and prints the result of the service request.

In [26]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: Waiting fot the wheel loader's response...")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
response = requests.get(url="https://broker.s3i.vswf.dev/"+sender_endpoint , headers=headers)
json_acceptable_string = response.text.replace("'", "\"")
response_json = json.loads(json_acceptable_string)
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: The wheel loader responded with:")
print(response_json["results"])

[S3I][2020-05-23 20:39:45]: Waiting fot the wheel loader's response...
[S3I][2020-05-23 20:39:46]: The wheel loader responded with:
I am moving stuff from A to B.


In [27]:
attributePath = input(
        '[S3I]: please enter the attribute path of the value you want to receive from the wheel loader (e.g., attributes/features/ml40::Composite/targets/ml40::Tank/features/ml40::LiquidFillingLevel/currentLevel): \n')

msg_uuid = "s3i:" + str(uuid.uuid4())

servReq = s3i.GetValueRequest()
servReq.fillGetValueRequest(senderUUID=clientId, receiverUUID=receivers, sender_endpoint=sender_endpoint,
                                attributePath=attributePath, msgUUID=msg_uuid)

[S3I]: please enter the attribute path of the value you want to receive from the wheel loader (e.g., attributes/features/ml40::Composite/targets/ml40::Tank/features/ml40::LiquidFillingLevel/currentLevel): 
attributes/features/ml40::Location


In [28]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: Sending the service request to the wheel loader")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
print(receiver_endpoints)
response = requests.post(url="https://broker.s3i.vswf.dev/"+receiver_endpoints[0] ,
                                data=json.dumps(servReq.msg), headers=headers)
print(response.text)

[S3I][2020-05-23 20:40:40]: Sending the service request to the wheel loader
['s3ibs://s3i:7f654f13-11ac-413d-841c-10a8e431fc35']
[{"endpoint": "s3ibs://s3i:7f654f13-11ac-413d-841c-10a8e431fc35", "status": "Sending message succeeded"}]


In [29]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: Waiting fot the wheel loader's response...")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
response = requests.get(url="https://broker.s3i.vswf.dev/"+sender_endpoint , headers=headers)
json_acceptable_string = response.text.replace("'", "\"")
response_json = json.loads(json_acceptable_string)
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: The wheel loader responded with:")
print(response_json["value"])

[S3I][2020-05-23 20:40:43]: Waiting fot the wheel loader's response...
[S3I][2020-05-23 20:40:43]: The wheel loader responded with:
{'sender': 's3i:7f654f13-11ac-413d-841c-10a8e431fc35', 'replyingToMessage': 's3i:912daed1-3951-4398-8a0d-0aefb50bd9f8', 'messageType': 'getValueReply', 'identifier': 's3i:6edf486e-1ce4-4155-a8e0-cc903ba7ba49', 'value': {'longitude': 7.97917, 'class': 'ml40::Location', 'latitude': 51.45278}, 'receivers': 's3i:a3afa256-b80c-4a6d-bd9c-786887c8ce4a'}
{'longitude': 7.97917, 'class': 'ml40::Location', 'latitude': 51.45278}
